# Pre-training experiments
This notebook contains various experiments with pre-training pipeline, inspired by MOP paper https://arxiv.org/abs/2109.04810

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd

## Prepare small graph

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/Code/mop/kg_dir/old/S20Rel"
PART_ENTITIES = -1

In [ ]:
cut_partitions = []
mapping = {}
with open(os.path.join(DATASET_DIR, "partition_20.txt")) as f:
  lines = f.readlines()
  for i in range(len(lines)):
    cut_partitions.append(lines[i].strip().split("\t")[:PART_ENTITIES])
    for idx in cut_partitions[-1]:
      mapping[idx] = i

In [ ]:
entities = []
with open(os.path.join(DATASET_DIR, "entity2id.txt")) as f:
  for line in f.readlines():
    for c in line.strip().split("\t"):
      if c.strip() in mapping:
        entities.append(line)

In [ ]:
len(entities)

In [ ]:
d = {}
with open(os.path.join(DATASET_DIR, "train2id.txt")) as f:
  count = int(f.readline().strip())
  print(count)
  for line in f.readlines():
    dat = line.strip().split("\t")
    if not dat[0] in d:
      d[dat[0]] = np.zeros(20)
    d[dat[0]][int(dat[2])] += 1

In [ ]:
np.mean(list(map(lambda x: np.max(x), d.values())))

In [ ]:
np.median(list(map(lambda x: np.max(x), d.values())))

In [ ]:
train_dummy = []
counts = np.zeros(len(cut_partitions))
with open(os.path.join(DATASET_DIR, "train2id.txt")) as f:
  lines = f.readlines()
  for i in range(1, len(lines)):
    ids = lines[i].strip().split("\t")
    if ids[0] in mapping and ids[1] in mapping and mapping[ids[0]] == mapping[ids[1]]:
      train_dummy.append(lines[i])
      counts[mapping[ids[0]]] += 1

In [ ]:
counts

In [ ]:
with open("entity2id.txt", "w") as f:
  f.write(str(len(entities)) + '\n' + "".join(entities))

with open("train2id.txt", "w") as f:
  f.write(str(len(train_dummy)) + '\n' + "".join(train_dummy))

In [ ]:
for i in range(len(cut_partitions)):
  cut_partitions[i] = "\t".join(cut_partitions[i])


with open("partition_20.txt", "w") as f:
  f.write("\n".join(cut_partitions))

## Dataset preparation

In [ ]:
from tqdm import tqdm
from cprint import cprint
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
import torch
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from datasets import Dataset

In [ ]:
# TODO: Simplify everything here, there is probably no need for both custom collate_fn and generator_fn
"""
Generator function for link prediction (for now, no negative sampling)
"""
def generator_fn_lp(data, id2ent, id2rel, entity_list):
  label = list(id2rel.keys()).index(data[2])
  return ((id2ent[data[0]], id2ent[data[1]]), label)

"""
Generator function for entity prediction
"""
def generator_fn_ep(data, id2ent, id2rel, entity_list):
  label = entity_list.index(data[1])
  return ((id2ent[data[0]], id2rel[data[2]]), label)


In [ ]:
"""
generator_fn(data, id2ent, id2rel, ent2part, partition_counts) - function that generates textual input and label from relation
Input:
- data - list [entityid_1, entityid_2, relationid]
- id2ent - dict {entityid: textual_description}
- id2rel - dict {relationshipid: textual_description}
- ent2part - dict {entityid: (partitionid, entityid_within_partition)}
- partition_counts - list, number of entities per partition
Output:
tuple (input_text, label)
"""
def prepare_data(dataset_path, tokenizer, generator_fn=None, include_border_rels=True, num_workers=2, max_seq_length=64):

  id2ent = {}
  cprint.info("Processing entities for graph")
  with open(os.path.join(dataset_path, "entity2id.txt")) as f:
    total_entities = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2ent[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  id2rel = {}
  cprint.info("Processing relations for graph")
  with open(os.path.join(dataset_path, "relation2id.txt")) as f:
    total_relations = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2rel[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  ent2part = {}
  total_groups = 0
  cprint.info("Processing partitions for graph")
  with open(os.path.join(dataset_path, "partition_20.txt")) as f:
    for group_idx, line in tqdm(enumerate(f.readlines())):
      for entid in line.strip().split("\t"):
        if not int(entid) in id2ent:
            cprint.err(f"Partition №{group_idx} has entity with invalid id {id}. Skipping...")
            continue
        if int(entid) in ent2part:
            cprint.err(f"Entity with ID {id} belongs to multiple partitions. Skipping...")
            continue
        ent2part[int(entid)] = group_idx
      total_groups += 1

  entities = [set([]) for i in range(total_groups)]
  cprint.info(f"Getting entities for partitions")
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
      count = int(f.readline())
      for line in tqdm(f.readlines()):
        data = list(map(int, line.strip().split("\t")))
        if not data[0] in ent2part or not data[1] in ent2part:
            cprint.err(f"No partition data available for triplet {data}. Skipping...")
            continue
        elif ent2part[data[0]] != ent2part[data[1]]:
            if include_border_rels:
                entities[ent2part[data[0]]].add(data[0])
                entities[ent2part[data[0]]].add(data[1])
                entities[ent2part[data[1]]].add(data[0])
                entities[ent2part[data[1]]].add(data[1])
            else:
                #cprint.err(f"Inter-partition link encountered: {data}. Skipping...")
                continue
        else:
            entities[ent2part[data[0]]].add(data[0])
            entities[ent2part[data[0]]].add(data[1])
            
  partition_counts = [len(entities[i]) for i in range(len(entities))]
  entity_list = [list(entities[i]) for i in range(len(entities))]  
  examples = [[] for i in range(total_groups)]
  cprint.info(f"Splitting edges into partitions. include_border_edges={include_border_rels}") 
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
    count = int(f.readline())
    for line in tqdm(f.readlines()):
      data = list(map(int, line.strip().split("\t")))
      if not data[0] in id2ent or not data[1] in id2ent:
        #cprint.err(f"No entity data available for triplet {data}. Skipping...")
        continue
      elif not data[0] in ent2part or not data[1] in ent2part:
        #cprint.err(f"No partition data available for triplet {data}. Skipping...")
        continue
      elif not data[2] in id2rel:
        #cprint.err(f"Invalid relation id encountered for triplet {data}. Skipping...")
        continue
      elif ent2part[data[0]] != ent2part[data[1]]:
        if include_border_rels:
          examples[ent2part[data[1]]].append(generator_fn(data, id2ent, id2rel, entity_list[ent2part[data[1]]]))
        else:
          #cprint.err(f"Inter-partition link encountered: {data}. Skipping...")
          continue
      examples[ent2part[data[0]]].append(generator_fn(data, id2ent, id2rel, entity_list[ent2part[data[0]]]))

  datasets = []
  cprint.info("Preparing datasets")
  for group in tqdm(examples):
      batch_pair_text = [example[0] for example in group]
      text_features = tokenizer.batch_encode_plus(
        batch_pair_text,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors="pt",
        truncation=True,
      )
      labels = torch.as_tensor([example[1] for example in group], dtype=torch.long)
      dataset = Dataset.from_dict({
        "input_ids": text_features.input_ids,
        "attention_mask": text_features.attention_mask,
        "token_type_ids": text_features.token_type_ids,
        "labels": labels,
      })
      dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
      datasets.append(dataset)
  return datasets, partition_counts, total_relations



In [ ]:
def prepare_data_full_lp(dataset_path, tokenizer, num_workers=2, max_seq_length=64):

  id2ent = {}
  cprint.info("Processing entities for graph")
  with open(os.path.join(dataset_path, "entity2id.txt")) as f:
    total_entities = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2ent[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  id2rel = {}
  cprint.info("Processing relations for graph")
  with open(os.path.join(dataset_path, "relation2id.txt")) as f:
    total_relations = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2rel[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  examples = []
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
    count = int(f.readline())
    for line in tqdm(f.readlines()):
      data = list(map(int, line.strip().split("\t")))
      if not data[0] in id2ent or not data[1] in id2ent:
        #cprint.err(f"No entity data available for triplet {data}. Skipping...")
        continue
      elif not data[2] in id2rel:
        #cprint.err(f"Invalid relation id encountered for triplet {data}. Skipping...")
        continue
      examples.append(((id2ent[data[0]], id2ent[data[1]]), list(id2rel.keys()).index(data[2])))

  batch_pair_text = [example[0] for example in examples]
  text_features = tokenizer.batch_encode_plus(
        batch_pair_text,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors="pt",
        truncation=True,
  )
  labels = torch.as_tensor([example[1] for example in examples], dtype=torch.long)
  dataset = Dataset.from_dict({
        "input_ids": text_features.input_ids,
        "attention_mask": text_features.attention_mask,
        "token_type_ids": text_features.token_type_ids,
        "labels": labels,
  })
  dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
  return dataset, total_relations

In [ ]:
def prepare_data_ep_mod(dataset_path, tokenizer, include_border_rels=True, num_workers=2, max_seq_length=64):

  id2ent = {}
  cprint.info("Processing entities for graph")
  with open(os.path.join(dataset_path, "entity2id.txt")) as f:
    total_entities = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2ent[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  id2rel = {}
  cprint.info("Processing relations for graph")
  with open(os.path.join(dataset_path, "relation2id.txt")) as f:
    total_relations = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2rel[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  ent2part = {}
  total_groups = 0
  cprint.info("Processing partitions for graph")
  with open(os.path.join(dataset_path, "partition_20.txt")) as f:
    for group_idx, line in tqdm(enumerate(f.readlines())):
      for entid in line.strip().split("\t"):
        if not int(entid) in id2ent:
            cprint.err(f"Partition №{group_idx} has entity with invalid id {id}. Skipping...")
            continue
        if int(entid) in ent2part:
            cprint.err(f"Entity with ID {id} belongs to multiple partitions. Skipping...")
            continue
        ent2part[int(entid)] = group_idx
      total_groups += 1

  entities = [set([]) for i in range(total_groups)]
  cprint.info(f"Getting entities for partitions")
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
      count = int(f.readline())
      for line in tqdm(f.readlines()):
        data = list(map(int, line.strip().split("\t")))
        if not data[0] in ent2part or not data[1] in ent2part:
            cprint.err(f"No partition data available for triplet {data}. Skipping...")
            continue
        elif ent2part[data[0]] != ent2part[data[1]]:
            if include_border_rels:
                entities[ent2part[data[0]]].add(data[0])
                entities[ent2part[data[0]]].add(data[1])
                entities[ent2part[data[1]]].add(data[0])
                entities[ent2part[data[1]]].add(data[1])
            else:
                #cprint.err(f"Inter-partition link encountered: {data}. Skipping...")
                continue
        else:
            entities[ent2part[data[0]]].add(data[0])
            entities[ent2part[data[0]]].add(data[1])
            
  partition_counts = [len(entities[i]) for i in range(len(entities))]
  entity_list = [list(entities[i]) for i in range(len(entities))]
  examples = [{} for i in range(total_groups)]  
  cprint.info(f"Splitting edges into partitions. include_border_edges={include_border_rels}") 
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
    count = int(f.readline())
    for line in tqdm(f.readlines()):
      data = list(map(int, line.strip().split("\t")))
      if not data[0] in id2ent or not data[1] in id2ent:
        #cprint.err(f"No entity data available for triplet {data}. Skipping...")
        continue
      elif not data[0] in ent2part or not data[1] in ent2part:
        #cprint.err(f"No partition data available for triplet {data}. Skipping...")
        continue
      elif not data[2] in id2rel:
        #cprint.err(f"Invalid relation id encountered for triplet {data}. Skipping...")
        continue
      elif ent2part[data[0]] != ent2part[data[1]]:
        if include_border_rels:
          partition = ent2part[data[1]]  
          if not (data[0], data[2]) in examples[partition]:
               examples[partition][(data[0], data[2])] = np.zeros(partition_counts[partition])
          examples[partition][(data[0], data[2])][entity_list[partition].index(data[1])] = 1
        else:
          #cprint.err(f"Inter-partition link encountered: {data}. Skipping...")
          continue
      partition = ent2part[data[0]]  
      if not (data[0], data[2]) in examples[partition]:
          examples[partition][(data[0], data[2])] = np.zeros(partition_counts[partition])
      examples[partition][(data[0], data[2])][entity_list[partition].index(data[1])] = 1

  datasets = []
  cprint.info("Preparing datasets")
  for group in tqdm(examples):
      batch_pair_text = [(id2ent[example[0]], id2ent[example[1]]) for example in group.keys()]
      labels = torch.as_tensor([group[example] for example in group.keys()], dtype=torch.long)
      text_features = tokenizer.batch_encode_plus(
        batch_pair_text,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors="pt",
        truncation=True,
      )
      dataset = Dataset.from_dict({
        "input_ids": text_features.input_ids,
        "attention_mask": text_features.attention_mask,
        "token_type_ids": text_features.token_type_ids,
        "labels": labels,
      })
      dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
      datasets.append(dataset)
      break
  return datasets, partition_counts, total_relations



In [ ]:
group_datasets_ep_mod_pubmedbert, part_ent_counts_ep, total_rel = prepare_data_ep_mod("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_pubmedbert, include_border_rels=True, num_workers=4)

Processing entities for graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232147/232147 [00:00<00:00, 1056305.58it/s]


Processing relations for graph


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 132521.45it/s]


Processing partitions for graph


20it [00:00, 132.99it/s]


Getting entities for partitions


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1125828/1125828 [00:02<00:00, 557001.50it/s]


Splitting edges into partitions. include_border_edges=True


 76%|███████████████████████████████████████████████████████████████████████████████████▊                          | 857710/1125828 [01:51<02:34, 1736.09it/s]

In [11]:
#full_lp_rel_scibert, total_rel = prepare_data_rel_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_scibert)
full_lp_rel_biobert, total_rel = prepare_data_rel_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_biobert)

Processing entities for graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232147/232147 [00:00<00:00, 1216471.09it/s]


Processing relations for graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1125828/1125828 [00:33<00:00, 34066.09it/s]


Preparing datasets


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [34:14<00:00, 102.74s/it]


In [10]:
full_lp_rel_pubmedbert, total_rel = prepare_data_rel_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_pubmedbert)

Processing entities for graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232147/232147 [00:00<00:00, 1050678.29it/s]


Processing relations for graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1125828/1125828 [00:43<00:00, 25953.63it/s]


Preparing datasets


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [35:18<00:00, 105.93s/it]


In [10]:
full_lp_scibert, total_rel = prepare_data_full_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_scibert)
full_lp_biobert, total_rel = prepare_data_full_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_biobert)
full_lp_pubmedbert, total_rel = prepare_data_full_lp("/vol/data/kg_dir/S20Rel", tokenizer=tokenizer_pubmedbert)

Processing entities for graph


100%|██████████████████████████████| 232147/232147 [00:00<00:00, 1202160.98it/s]


Processing relations for graph


100%|█████████████████████████████| 1125828/1125828 [00:02<00:00, 399722.42it/s]


Processing entities for graph


...

100%|█████████████████████████████| 1125828/1125828 [00:02<00:00, 394320.93it/s]


Processing entities for graph


100%|██████████████████████████████| 232147/232147 [00:00<00:00, 1152992.31it/s]


Processing relations for graph


100%|█████████████████████████████| 1125828/1125828 [00:02<00:00, 400559.56it/s]


In [7]:
def prepare_data_rel_lp(dataset_path, tokenizer, num_workers=2, max_seq_length=64):

  id2ent = {}
  cprint.info("Processing entities for graph")
  with open(os.path.join(dataset_path, "entity2id.txt")) as f:
    total_entities = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2ent[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

  id2rel = {}
  cprint.info("Processing relations for graph")
  with open(os.path.join(dataset_path, "relation2id.txt")) as f:
    total_relations = int(f.readline().strip())
    for line in tqdm(f.readlines()):
      id2rel[int(line.strip().split("\t")[1])] = line.strip().split("\t")[0]

            
  examples = [[] for i in range(total_relations)]
  with open(os.path.join(dataset_path, "train2id.txt")) as f:
    count = int(f.readline())
    for line in tqdm(f.readlines()):
      data = list(map(int, line.strip().split("\t")))
      if not data[0] in id2ent or not data[1] in id2ent:
        #cprint.err(f"No entity data available for triplet {data}. Skipping...")
        continue
      elif not data[2] in id2rel:
        #cprint.err(f"Invalid relation id encountered for triplet {data}. Skipping...")
        continue
      for i in range(total_relations):
          examples[i].append(((id2ent[data[0]], id2ent[data[1]]), int(i == list(id2rel.keys()).index(data[2]))))

  datasets = []
  cprint.info("Preparing datasets")
  for group in tqdm(examples):
      batch_pair_text = [example[0] for example in group]
      text_features = tokenizer.batch_encode_plus(
        batch_pair_text,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors="pt",
        truncation=True,
      )
      labels = torch.as_tensor([example[1] for example in group], dtype=torch.long)
      dataset = Dataset.from_dict({
        "input_ids": text_features.input_ids,
        "attention_mask": text_features.attention_mask,
        "token_type_ids": text_features.token_type_ids,
        "labels": labels,
      })
      dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
      datasets.append(dataset)
  return datasets, total_relations



In [9]:
tokenizer_pubmedbert = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext") 
tokenizer_biobert = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1") 
tokenizer_scibert = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
group_datasets_ep_pubmedbert, part_ent_counts_ep, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_ep,
                                tokenizer=tokenizer_pubmedbert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=True)

In [ ]:
group_datasets_ep_biobert, part_ent_counts_ep, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_ep,
                                tokenizer=tokenizer_biobert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=True)

In [ ]:
group_datasets_ep_scibert, part_ent_counts_ep, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_ep,
                                tokenizer=tokenizer_scibert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=True)

In [ ]:
group_datasets_lp_no_border_biobert, part_ent_counts_lp_nb, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_lp,
                                tokenizer=tokenizer_biobert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=False)

In [ ]:
group_datasets_lp_no_border_scibert, part_ent_counts_lp_nb, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_lp,
                                tokenizer=tokenizer_scibert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=False)

In [ ]:
group_datasets_ep_no_border_biobert, part_ent_counts_ep_nb, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_ep,
                                tokenizer=tokenizer_biobert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=False)

In [ ]:
group_datasets_ep_no_border_scibert, part_ent_counts_ep_nb, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_ep,
                                tokenizer=tokenizer_scibert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=False)

In [ ]:
group_datasets_lp_biobert, part_ent_counts_lp, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_lp,
                                tokenizer=tokenizer_biobert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=True)

In [ ]:
group_datasets_lp_scibert, part_ent_counts_lp, total_rel = prepare_data("/vol/data/kg_dir/S20Rel", 
                                generator_fn=generator_fn_lp,
                                tokenizer=tokenizer_scibert,
                                num_workers=4, 
                                max_seq_length=64, 
                                include_border_rels=True)

## Pre-training

In [12]:
import torch
import random
from adapters import SeqBnConfig
from adapters import AutoAdapterModel
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer
import wandb
import datetime
from torch.nn import CrossEntropyLoss

/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [13]:
os.environ["WANDB_PROJECT"] = "Pre-Training" 
os.environ["WANDB_LOG_MODEL"] = ""

In [14]:
GLOBAL_ACC = []

In [15]:
class WeightedLossAdapterTrainer(AdapterTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        class_weights = torch.tensor([0.1, 0.9]).to(logits.device)  # Adjust based on your dataset
        loss_fct = CrossEntropyLoss(weight=class_weights)
        
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [16]:
def init_model(num_labels, seed, adapter_name, model_name):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  base_config = AutoConfig.from_pretrained(model_name)
  model = AutoAdapterModel.from_pretrained(model_name, config=base_config)
  adapter_config = SeqBnConfig(reduction_factor=8)
  model.add_adapter(adapter_name, config=adapter_config)
  model.add_classification_head(adapter_name, num_labels=num_labels)
  model.train_adapter(adapter_name)
  model.set_active_adapters(adapter_name)
  model.to(device)
  return model

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean(), }

def train_group(group_dataset, num_labels, training_args, adapter_name, model_name, seed=1):
    model = init_model(num_labels=num_labels, seed=seed, adapter_name=adapter_name, model_name=model_name)
    if num_labels == 2:
        trainer = WeightedLossAdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=group_dataset,
            eval_dataset=group_dataset,
            compute_metrics=compute_metrics,
        )
    else:
        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=group_dataset,
            eval_dataset=group_dataset,
            compute_metrics=compute_metrics,
        )
    train_res = trainer.train()
    wandb.log({"train_res": train_res})
    print(train_res)
    eval_res = trainer.evaluate()
    wandb.log({"eval_res": eval_res})
    print(eval_res)
    return model

def train_dataset(save_path, datasets, adapter_name, model_name, training_args, num_classes, seed=1, start_idx=0):
    assert len(datasets) == len(num_classes)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for idx in tqdm(range(start_idx, len(datasets))):
        partition = datasets[idx]
        cprint.info(f"Training adapter for partition {idx}")
        training_args.run_name = adapter_name + f"_partition_{idx}_epoch_{training_args.num_train_epochs}_" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        model = train_group(group_dataset=partition, 
                            num_labels=num_classes[idx], 
                            training_args=training_args, 
                            adapter_name=adapter_name, 
                            model_name=model_name, 
                            seed=seed)
        cprint.info(f"Saving adapter to {os.path.join(save_path, training_args.run_name)}")
        model.save_adapter(os.path.join(save_path, training_args.run_name), adapter_name)
        wandb.finish()
        del model
        torch.cuda.empty_cache()

In [17]:
training_args_scibert_pubmedbert = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=1,
        logging_steps=20,
        output_dir="/vol/data/mkonov_output",
        overwrite_output_dir=True,
        weight_decay=0.01,
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        #report_to=[]
        report_to="wandb",
    )

training_args_biobert = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=2,
        logging_steps=20,
        output_dir="/vol/data/mkonov_output",
        overwrite_output_dir=True,
        weight_decay=0.01,
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        #report_to=[]
        report_to="wandb",
    )

In [18]:
torch.cuda.empty_cache()

In [19]:
wandb.finish()

In [38]:
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR API key must be 40 characters long, yours was 48
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR API key must be 40 characters long, yours was 48
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [19]:
train_dataset(save_path="/vol/data/adapters/PubMedBERT/S20Rel_LP_full", 
              datasets=[full_lp_pubmedbert], 
              num_classes=[total_rel],
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="pubmedbert_S20Rel_LP_full", 
              model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

  0%|                                                     | 0/1 [00:00<?, ?it/s]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 0


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: konovma (SoSe2024-NLP-Lab). Use `wandb login --relogin` to force relogin


Step,Training Loss
20,2.785200
40,2.448800
60,1.874100
80,1.328300
100,0.939100
120,0.658100
140,0.488100
160,0.385100
180,0.333900
200,0.297800


TrainOutput(global_step=4398, training_loss=0.14737696604817604, metrics={'train_runtime': 2048.0233, 'train_samples_per_second': 549.714, 'train_steps_per_second': 2.147, 'total_flos': 3.832717137972941e+16, 'train_loss': 0.14737696604817604, 'epoch': 1.0})


{'eval_loss': 0.06044953688979149, 'eval_acc': 0.9729914338602345, 'eval_runtime': 1040.5788, 'eval_samples_per_second': 1081.925, 'eval_steps_per_second': 4.226, 'epoch': 1.0}
Saving adapter to /vol/data/adapters/PubMedBERT/S20Rel_LP_full/pubmedbert_S20Rel_LP_full_partition_0_epoch_1_2024-07-10 00:06:12


eval/acc,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▄▅█▄▃▃▆▄▄▄▃▃▂▂▃▃▆▂▃▃▂▄▁▃▃▃▂▁▂▄▃▂▃▂▃▁▅▁▁▄
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/acc,0.97299


100%|███████████████████████████████████████████| 1/1 [51:38<00:00, 3098.27s/it]


In [20]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_LP_full", 
              datasets=[full_lp_biobert], 
              num_classes=[total_rel],
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_LP_full", 
              model_name="dmis-lab/biobert-v1.1")

  0%|                                                     | 0/1 [00:00<?, ?it/s]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 0


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
20,2.741600
40,2.339200
60,1.606900
80,0.994500
100,0.635600
120,0.454400
140,0.371500
160,0.308400
180,0.294900
200,0.270300


TrainOutput(global_step=8796, training_loss=0.09819799975380675, metrics={'train_runtime': 4096.2044, 'train_samples_per_second': 549.693, 'train_steps_per_second': 2.147, 'total_flos': 7.611597721383322e+16, 'train_loss': 0.09819799975380675, 'epoch': 2.0})


{'eval_loss': 0.046076368540525436, 'eval_acc': 0.9779362389281488, 'eval_runtime': 1042.819, 'eval_samples_per_second': 1079.601, 'eval_steps_per_second': 4.217, 'epoch': 2.0}
Saving adapter to /vol/data/adapters/BioBERT/S20Rel_LP_full/biobert_S20Rel_LP_full_partition_0_epoch_2_2024-07-10 00:57:51


eval/acc,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▅▅▅▅▄▄▄▅▂▃▃▄▂▃▃▃▂▄▃▂▃▁▂█▄▃▄▃▁▂▂▃▂▂▃▄▁▂▃▂
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/acc,0.97794


100%|█████████████████████████████████████████| 1/1 [1:25:47<00:00, 5147.75s/it]


In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_LP_full", 
              datasets=[full_lp_scibert], 
              num_classes=[total_rel],
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="scibert_S20Rel_LP_full", 
              model_name="allenai/scibert_scivocab_uncased")

  0%|                                                     | 0/1 [00:00<?, ?it/s]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 0


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
20,2.589500
40,1.836100
60,1.072200
80,0.638600
100,0.430700
120,0.332600
140,0.282600
160,0.249900
180,0.224100
200,0.213300


In [20]:
train_dataset(save_path="/vol/data/adapters/PubMedBERT/S20Rel_LP_Rel", 
              datasets=full_lp_rel_pubmedbert, 
              num_classes=[2] * total_rel,
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="pubmedbert_S20Rel_LP_Rel", 
              model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
              start_idx=9)

  0%|                                                                                                                                  | 0/11 [00:00<?, ?it/s]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 9


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: konovma (SoSe2024-NLP-Lab). Use `wandb login --relogin` to force relogin


Step,Training Loss
20,0.626900
40,0.474900
60,0.308600
80,0.208000
100,0.153300
120,0.108200
140,0.113700
160,0.111800
180,0.079800
200,0.085300


TrainOutput(global_step=4398, training_loss=0.040862478739351185, metrics={'train_runtime': 2046.6109, 'train_samples_per_second': 550.094, 'train_steps_per_second': 2.149, 'total_flos': 3.832118723463782e+16, 'train_loss': 0.040862478739351185, 'epoch': 1.0})


...

Step,Training Loss
20,0.534000
40,0.189900
60,0.082300
80,0.042800
100,0.033500
120,0.029700
140,0.014300
160,0.023900
180,0.013300
200,0.018200


TrainOutput(global_step=4398, training_loss=0.008720280402778339, metrics={'train_runtime': 2051.7582, 'train_samples_per_second': 548.714, 'train_steps_per_second': 2.144, 'total_flos': 3.832118723463782e+16, 'train_loss': 0.008720280402778339, 'epoch': 1.0})


{'eval_loss': 0.0012482249876484275, 'eval_acc': 0.9994901530251513, 'eval_runtime': 1045.3945, 'eval_samples_per_second': 1076.941, 'eval_steps_per_second': 4.207, 'epoch': 1.0}
Saving adapter to /vol/data/adapters/PubMedBERT/S20Rel_LP_Rel/pubmedbert_S20Rel_LP_Rel_partition_19_epoch_1_2024-07-11 06:26:05


eval/acc,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▃▂▂▂▃█▁▂▂▂▃▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▃▁▁▁
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁
eval/acc,0.99949


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [9:28:17<00:00, 3099.74s/it]


In [ ]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_LP_Rel", 
              datasets=full_lp_rel_biobert, 
              num_classes=[2] * total_rel,
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_LP_Rel", 
              model_name="dmis-lab/biobert-v1.1")

  0%|                                                                                                                                  | 0/20 [00:00<?, ?it/s]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 0


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
20,0.377300
40,0.190500
60,0.066100
80,0.013400
100,0.004900
120,0.005600
140,0.001100
160,0.000300
180,0.001600
200,0.000600


TrainOutput(global_step=8796, training_loss=0.0016352417855824168, metrics={'train_runtime': 4101.1415, 'train_samples_per_second': 549.032, 'train_steps_per_second': 2.145, 'total_flos': 7.610400892365005e+16, 'train_loss': 0.0016352417855824168, 'epoch': 2.0})


...

 10%|███████████▌                                                                                                       | 2/20 [2:51:58<25:48:10, 5160.59s/it]/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training adapter for partition 2


/vol/data/.virtualenv/nlp-lab/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
20,0.492100
40,0.143400
60,0.032400
80,0.032100
100,0.016900
120,0.015600
140,0.006700
160,0.017300
180,0.022100
200,0.013000


TrainOutput(global_step=8796, training_loss=0.0038998152203057975, metrics={'train_runtime': 4102.9596, 'train_samples_per_second': 548.788, 'train_steps_per_second': 2.144, 'total_flos': 7.610400892365005e+16, 'train_loss': 0.0038998152203057975, 'epoch': 2.0})


In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_LP_Rel", 
              datasets=full_lp_rel_scibert, 
              num_classes=[2] * total_rel,
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="scibert_S20Rel_LP_Rel", 
              model_name="allenai/scibert_scivocab_uncased")

In [ ]:
train_dataset(save_path="/vol/data/adapters/PubMedBERT/S20Rel_EP", 
              datasets=group_datasets_ep_pubmedbert, 
              num_classes=part_ent_counts_ep,
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="pubmedbert_S20Rel_EP", 
              model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [ ]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_EP", 
              datasets=group_datasets_ep_biobert, 
              num_classes=part_ent_counts_ep,
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_EP", 
              model_name="dmis-lab/biobert-v1.1")

In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_EP", 
              datasets=group_datasets_ep_scibert, 
              num_classes=part_ent_counts_ep,
              training_args=training_args_scibert_pubmedbert, 
              adapter_name="scibert_S20Rel_EP", 
              model_name="allenai/scibert_scivocab_uncased")

In [ ]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_LP", 
              datasets=group_datasets_lp_biobert, 
              num_classes=part_ent_counts_lp,
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_LP", 
              model_name="dmis-lab/biobert-v1.1")

In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_LP", 
              datasets=group_datasets_lp_scibert, 
              num_classes=part_ent_counts_lp,
              training_args=training_args_scibert, 
              adapter_name="scibert_S20Rel_LP", 
              model_name="allenai/scibert_scivocab_uncased")

In [ ]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_LP_NB", 
              datasets=group_datasets_lp_no_border_biobert, 
              num_classes=part_ent_counts_lp_nb,
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_LP_NB", 
              model_name="dmis-lab/biobert-v1.1")

In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_LP_NB", 
              datasets=group_datasets_lp_no_border_scibert, 
              num_classes=part_ent_counts_lp_nb,
              training_args=training_args_scibert, 
              adapter_name="scibert_S20Rel_LP_NB", 
              model_name="allenai/scibert_scivocab_uncased")

In [ ]:
train_dataset(save_path="/vol/data/adapters/BioBERT/S20Rel_EP_NB", 
              datasets=group_datasets_ep_no_border_biobert, 
              num_classes=part_ent_counts_ep_nb,
              training_args=training_args_biobert, 
              adapter_name="biobert_S20Rel_EP_NB", 
              model_name="dmis-lab/biobert-v1.1")

In [ ]:
train_dataset(save_path="/vol/data/adapters/SciBERT/S20Rel_EP_NB", 
              datasets=group_datasets_ep_no_border_scibert, 
              num_classes=part_ent_counts_ep_nb,
              training_args=training_args_scibert, 
              adapter_name="scibert_S20Rel_EP_NB", 
              model_name="allenai/scibert_scivocab_uncased")